In [1]:
import ssl
import json
import asyncio
import logging
import websockets
import threading 
from smbus2 import SMBus, i2c_msg 

bus = SMBus(1) 
ws_conns = set() 

async def broadcaster(ws_conn):
    global ws_conns 
    try: 
        ws_conns.add(ws_conn)
        # Send current state to user
        websockets.broadcast(ws_conns, json.dumps({"type": "users", "count": len(ws_conns)}))

        async for msg in ws_conn:
            websockets.broadcast(ws_conns, msg)
            msg   = json.loads(msg)
            dev   = msg["device"]
            val   = msg["value"]
            val_t = msg["value_type"]  
            if dev=='keyboard':
                if val_t == "string":
                    val = ord(val)  
                else:
                    val = int(val)
                bus.i2c_rdwr(i2c_msg.write(0x34, [ord('k'), ord('w'), val, 255, 5]))
            print(msg) 
            # # bytes must be in range [0, 256)
    finally: 
        ws_conns.remove(ws_conn)
        websockets.broadcast(ws_conns, json.dumps({"type": "users", "count": len(ws_conns)}))

async def main():
    #ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER) 
    #ssl_context.load_cert_chain("/path/to/the/cert_chain") 
    async with websockets.serve(broadcaster, "192.168.12.238", 8765): # , ssl=ssl_context):
        await asyncio.Future()  # run forever 

def foo(): asyncio.run(main())
threading.Thread(target=foo).start()  

In [3]:
bus.close()